In [ ]:
#| default_exp pymor.operators

# pymor.operators

> Extended functionality for [pyMOR](https://pymor.org/) operators

In [ ]:
#| export
from __future__ import annotations
from pathlib import Path
import importlib
from numbers import Number
import math
from functools import reduce

from fastcore.basics import patch
from fastcore.meta import delegates
from pydash import unzip

import numpy as np
from numpy import array, ndarray
from scipy.sparse import issparse, sparray
from sparse import SparseArray
import sympy as sy
import pandas as pd
from xarray import DataArray, Coordinates

from pymor.basic import LincombOperator, NumpyMatrixOperator, ZeroOperator, ExpressionParameterFunctional, NumpyVectorSpace, Mu, IdentityOperator
from pymor.models.interface import Model
from pymor.algorithms.to_matrix import to_matrix, ToMatrixRules
from pymor.algorithms.simplify import ExpandRules, expand, ContractRules, contract
from pymor.algorithms.rules import match_class, RuleTable
from pymor.algorithms.lincomb import assemble_lincomb
from pymor.operators.interface import Operator
from pymor.core.base import abstractmethod
from pymor.operators.interface import Operator
from pymor.parameters.functionals import ParameterFunctional

from pylgs.patches import *
from pylgs.utilities.xarray import *
from pylgs.utilities.sparse import sparse, sparse2d, kron, sparse2d_rand
from pylgs.pymor.parameters import *
from pylgs.pymor.vectorarrays import *

In [ ]:
# | hide
from fastcore.test import test_eq, test_close
from pylgs.utilities.testing import test_array

## XarrayMatrixOperator

`XarrayMatrixOperator` is a replacement for `NumpyMatrixOperator` that affords several advantages.

In [ ]:
#| export
def _mapping_str(op:Operator):
    return f"{op.source.short_str()} → {op.range.short_str()}"

In [ ]:
#| exporti
class XarrayMatrixBasedOperator(Operator):
    """Base class for operators that assemble into an `XarrayMatrixOperator`."""

    linear = True

    @property
    def H(self):
        if not self.parametric:
            return self.assemble().H
        else:
            return super().H

    @abstractmethod
    def assemble(self, mu=None): pass

    def apply(self, U, mu=None):
        return self.range.from_xarray(self.assemble(mu).matrix.dot(U.impl._array))

    def apply_adjoint(self, V, mu=None):
        return self.assemble(mu).apply_adjoint(V)

    def as_range_array(self, mu=None):
        return self.range.from_xarray(self.assemble(mu).matrix.copy())

    def as_source_array(self, mu=None):
        return self.source.from_xarray(self.assemble(mu).matrix.copy()).conj()

    @property
    def display_name(self):
        return self.name if self.name != self.__class__.__name__ else 'X'

    def __str__(self):
        return f"{self.display_name}{_mapping_str(self)}"

    def _repr_html_(self):
        return str(self)

### _ndims -


In [ ]:
#| export
def _ndims(matrix):
    if isinstance(matrix, DataArray):
        return len(matrix.dims)
    else:
        return matrix.ndim    

### _get_space_dims -


In [ ]:
#| export
def _get_space_dims(dims, matrix, default):
    if dims is None:
        if hasattr(matrix, 'dims'): return list(matrix.dims)
        else: return [default]
    elif isinstance(dims, str): return [dims]
    elif hasattr(dims, 'dims'): return dims.dims
    return list(dims)

### XarrayMatrixOperator -

In [ ]:
#| export
class XarrayMatrixOperator(XarrayMatrixBasedOperator):
    """An `Operator` backed by an xarray `DataArray`."""

    def __init__(
        self, 
        matrix:DataArray|ndarray|SparseArray|sparray, # N-dimensional matrix
        source:XarrayVectorSpace|Coordinates|dict|list|str=None, # Source vector space, coordinates, or dimension name(s)
        range:XarrayVectorSpace|Coordinates|dict|list|str=None, # Range vector space, coordinates, or dimension name(s)
        solver_options:dict=None, # Options for matrix solver
        name:str=None # Operator name
    ):
        if (source is None or range is None) and _ndims(matrix) != 2: 
            raise ValueError(f'source and range must be specified if array {matrix} does not have two dimensions.')
        if issparse(matrix): matrix = sparse(matrix)
        range_dims = _get_space_dims(range, matrix[:, 0], 'range')
        source_dims = _get_space_dims(source, matrix[0], 'source')
        if not isinstance(matrix, DataArray):
            matrix = DataArray(
                matrix, 
                dims=range_dims + source_dims, 
                coords=getattr(range, 'coords', Coordinates()) + getattr(source, 'coords', Coordinates()),
                name=name
            )
        if name is None: name = matrix.name
        source = XarrayVectorSpace({dim: matrix[dim] for dim in source_dims})
        range = XarrayVectorSpace({dim: matrix[dim] for dim in range_dims}, name=name)
        self.__auto_init(locals())

    def assemble(self, mu=None): return self

    @property
    def size(self):
        return self.matrix.size
        
    @property
    def sparse(self):
        return isinstance(self.matrix.data, SparseArray)

A pyMOR `Operator` maps a `VectorArray` from a source `VectorSpace` to another `VectorArray` in a range `VectorSpace`. The `XarrayMatrixOperator` acts on an `XarrayVectorArray` from an `XarrayVectorSpace` by means of multiplication by an xarray `DataArray`.

The `html` representation of an `XarrayMatrixOperator` is *operator name*{*source space*} → {*range space*}, where X is used as the name if the operator has not been given an explict name.

The matrix can be supplied as a `DataArray`, or as an unlabeled numpy `ndarray`, scipy.sparse `sparray`, or sparse `SparseArray`. 

Create from a numpy array with default range and source:

In [ ]:
op = XarrayMatrixOperator(np.ones((2, 2)))
op

XarrayMatrixOperator(
    <xarray.DataArray (range: 2, source: 2)> Size: 32B
    array([[1., 1.],
           [1., 1.]])
    Dimensions without coordinates: range, source,
    source=XarrayVectorSpace(
               coords=Coordinates:
                        * source   (source) int64 16B 0 1),
    range=XarrayVectorSpace(
              coords=Coordinates:
                       * range    (range) int64 16B 0 1))

The `DataArray` backing the `Operator` is stored in the `matrix` attribute:

In [ ]:
op.matrix

<xarray.DataArray (range: 2, source: 2)> Size: 32B
array([[1., 1.],
       [1., 1.]])
Dimensions without coordinates: range, source

The `source` and `range` attributes store the corresponding `VectorArray`s:

In [ ]:
op.source

XarrayVectorSpace(
    coords=Coordinates:
             * source   (source) int64 16B 0 1)

In [ ]:
op.range

XarrayVectorSpace(
    coords=Coordinates:
             * range    (range) int64 16B 0 1)

Create from a numpy array with specified range and source names:

In [ ]:
XarrayMatrixOperator(array([[1, 0], [0, 1]]), source='source a', range='range a')

XarrayMatrixOperator(
    <xarray.DataArray (range a: 2, source a: 2)> Size: 32B
    array([[1, 0],
           [0, 1]])
    Dimensions without coordinates: range a, source a,
    source=XarrayVectorSpace(
               coords=Coordinates:
                        * source a  (source a) int64 16B 0 1),
    range=XarrayVectorSpace(
              coords=Coordinates:
                       * range a  (range a) int64 16B 0 1))

Create from a numpy array and specified range and source spaces:

In [ ]:
space_a = XarrayVectorSpace({'A': np.linspace(0, 1, 2)})
space_b = XarrayVectorSpace({'B': ['c', 'd', 'e']})
XarrayMatrixOperator(np.arange(6).reshape(2, 3), source=space_b, range=space_a, name='foo')

XarrayMatrixOperator(
    <xarray.DataArray 'foo' (A: 2, B: 3)> Size: 48B
    array([[0, 1, 2],
           [3, 4, 5]])
    Coordinates:
      * A        (A) float64 16B 0.0 1.0
      * B        (B) <U1 12B 'c' 'd' 'e',
    source=XarrayVectorSpace(
               coords=Coordinates:
                        * B        (B) <U1 12B 'c' 'd' 'e'),
    range=XarrayVectorSpace(
              coords=Coordinates:
                       * A        (A) float64 16B 0.0 1.0,
              name='foo'),
    name='foo')

In [ ]:
_.matrix

<xarray.DataArray 'foo' (A: 2, B: 3)> Size: 48B
array([[0, 1, 2],
       [3, 4, 5]])
Coordinates:
  * A        (A) float64 16B 0.0 1.0
  * B        (B) <U1 12B 'c' 'd' 'e'

Create from a `DataArray` with defined coords:

In [ ]:
XarrayMatrixOperator(
    DataArray(
        np.arange(6).reshape(2, 3), 
        {'A': np.linspace(0, 1, 2), 'B': np.linspace(0, 1, 3)}
    )
)

XarrayMatrixOperator(
    <xarray.DataArray (A: 2, B: 3)> Size: 48B
    array([[0, 1, 2],
           [3, 4, 5]])
    Coordinates:
      * A        (A) float64 16B 0.0 1.0
      * B        (B) float64 24B 0.0 0.5 1.0,
    source=XarrayVectorSpace(
               coords=Coordinates:
                        * B        (B) float64 24B 0.0 0.5 1.0),
    range=XarrayVectorSpace(
              coords=Coordinates:
                       * A        (A) float64 16B 0.0 1.0))

Create from a DataArray array with default coordinate names:

In [ ]:
op = XarrayMatrixOperator(DataArray(np.ones((2, 2)), name='foo'))
op

XarrayMatrixOperator(
    <xarray.DataArray 'foo' (dim_0: 2, dim_1: 2)> Size: 32B
    array([[1., 1.],
           [1., 1.]])
    Dimensions without coordinates: dim_0, dim_1,
    source=XarrayVectorSpace(
               coords=Coordinates:
                        * dim_1    (dim_1) int64 16B 0 1),
    range=XarrayVectorSpace(
              coords=Coordinates:
                       * dim_0    (dim_0) int64 16B 0 1,
              name='foo'),
    name='foo')

In [ ]:
op = XarrayMatrixOperator(sparse2d_rand([4, 5], density=.5, random_state=42))
op

XarrayMatrixOperator(
    <xarray.DataArray (range: 4, source: 5)> Size: 240B
    <COO: shape=(4, 5), dtype=float64, nnz=10, fill_value=0.0>
    Dimensions without coordinates: range, source,
    source=XarrayVectorSpace(
               coords=Coordinates:
                        * source   (source) int64 40B 0 1 2 3 4),
    range=XarrayVectorSpace(
              coords=Coordinates:
                       * range    (range) int64 32B 0 1 2 3))

### H -


In [ ]:
#| export
@patch(as_prop=True)
def H(self:XarrayMatrixOperator):
    """The adjoint operator."""
    adjoint_matrix = self.matrix if np.isrealobj(self.matrix) else self.matrix.conj()           
    options = {'inverse': self.solver_options.get('inverse_adjoint'),
               'inverse_adjoint': self.solver_options.get('inverse')} if self.solver_options else None
    return self.with_(matrix=adjoint_matrix, source=self.range, range=self.source, 
                      solver_options=options, name=self.name + '_adjoint')

In [ ]:
op.H

XarrayMatrixOperator(
    <xarray.DataArray (range: 4, source: 5)> Size: 240B
    <COO: shape=(4, 5), dtype=float64, nnz=10, fill_value=0.0>
    Dimensions without coordinates: range, source,
    source=XarrayVectorSpace(
               coords=Coordinates:
                        * range    (range) int64 32B 0 1 2 3),
    range=XarrayVectorSpace(
              coords=Coordinates:
                       * source   (source) int64 40B 0 1 2 3 4,
              name='XarrayMatrixOperator_adjoint'),
    name='XarrayMatrixOperator_adjoint')

### apply -


In [ ]:
#| export
@patch
def apply(
    self:XarrayMatrixOperator, 
    U, # `XarrayVectorArray` of vectors to which the operator is applied
    mu=None # The parameter values for which to evaluate the operator
)->XarrayVectorArray: # `XarrayVectorArray` in the range `XarrayVectorSpace`
    """Apply the operator to an `XarrayVectorArray` in the source vector space."""
    U = self.range.make_array(self.matrix.dot(U.array))
    if self.name != self.__class__.__name__: U = U.rename(self.name)
    return U

### apply_adjoint -

In [ ]:
#| export
@patch
def apply_adjoint(
    self:XarrayMatrixOperator, 
    V, # `XarrayVectorArray` of vectors to which the operator is applied
    mu=None # The parameter values for which to evaluate the operator
)->XarrayVectorArray: # `XarrayVectorArray` in the source `XarrayVectorSpace`
    """Apply the adjoint of the operator to an `XarrayVectorArray` in the range vector space."""
    return self.H.apply(V, mu=mu)

In [ ]:
V = op.range.zeros({'len': [0]})

In [ ]:
op.apply_adjoint(V)

XarrayVectorArray(
    XarrayVectorSpace(
        coords=Coordinates:
                 * source   (source) int64 40B 0 1 2 3 4,
        name='XarrayMatrixOperator_adjoint'),
    XarrayVectorArrayImpl(
        <xarray.DataArray 'XarrayMatrixOperator_adjoint' (source: 5, len: 1)> Size: 40B
        array([[0.],
               [0.],
               [0.],
               [0.],
               [0.]])
        Coordinates:
          * len      (len) int64 8B 0
        Dimensions without coordinates: source,
        <xarray.DataArray 'XarrayMatrixOperator_adjoint' (source: 5)> Size: 40B
        array([0., 0., 0., 0., 0.])
        Coordinates:
          * source   (source) int64 40B 0 1 2 3 4),
    _len=1)

### _assemble_lincomb -

In [ ]:
#| export
@patch
def _assemble_lincomb(self:XarrayMatrixOperator, operators, coefficients, identity_shift=0., solver_options=None, name=None):
    if not all(isinstance(op, XarrayMatrixOperator) for op in operators):
        return None

    common_mat_dtype = reduce(np.promote_types,
                              (op.matrix.dtype for op in operators if hasattr(op, 'matrix')))
    common_coef_dtype = reduce(np.promote_types, (type(c) for c in coefficients + [identity_shift]))
    common_dtype = np.promote_types(common_mat_dtype, common_coef_dtype)

    if coefficients[0] == 1:
        matrix = operators[0].matrix.astype(common_dtype)
    else:
        matrix = operators[0].matrix * coefficients[0]
        if matrix.dtype != common_dtype:
            matrix = matrix.astype(common_dtype)

    for op, c in zip(operators[1:], coefficients[1:]):
        if c == 1:
            try:
                matrix += op.matrix
            except NotImplementedError:
                matrix = matrix + op.matrix
        elif c == -1:
            try:
                matrix -= op.matrix
            except NotImplementedError:
                matrix = matrix - op.matrix
        else:
            try:
                matrix += (op.matrix * c)
            except NotImplementedError:
                matrix = matrix + (op.matrix * c)

    if identity_shift: raise NotImplementedError

    return XarrayMatrixOperator(matrix, source=self.source, range=self.range, solver_options=solver_options)

In [ ]:
#| hide
LincombOperator([op, op], [.3, .456]).assemble().matrix.data.todense()

array([[5.88746976e-04, 0.00000000e+00, 5.34212675e-03, 0.00000000e+00,
        7.36159172e-01],
       [4.62409789e-01, 1.74351933e-02, 0.00000000e+00, 0.00000000e+00,
        7.50111939e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        3.52792414e-02],
       [3.02294895e-01, 0.00000000e+00, 3.96729643e-01, 4.66816021e-01,
        0.00000000e+00]])

### to_matrix_XarrayMatrixOperator -


In [ ]:
#| exporti
@match_class(XarrayMatrixOperator)
def to_matrix_XarrayMatrixOperator(self, op):
    """Return the operator as a 2D matrix with stacked range and source dimensions in either `ndarray` or `sparray` format."""
    data = op.matrix.stack(_range=op.range.coords.dims, _source=op.source.coords.dims).data
    if isinstance(data, SparseArray): return sparse2d(data)
    return data
    
ToMatrixRules.insert_rule(-1, to_matrix_XarrayMatrixOperator)

In [ ]:
#| hide
to_matrix(op)

<Compressed Sparse Row sparse array of dtype 'float64'
	with 10 stored elements and shape (4, 5)>

In [ ]:
#| export
@patch
def to_numpy(self:XarrayMatrixOperator):
    """Return the operator in dense `ndarray` format."""
    return self.matrix.to_numpy()

The returned numpy array will be of shape $r_1\times \ldots \times r_n \times s_1 \times \ldots \times s_m$ where $r_i$ are the dimensions of the $n$ vector spaces making up the range product vector space, and $s_j$ are the dimensions of the $m$ vector spaces making up the source product vector space.

In [ ]:
op.to_numpy()

array([[7.78765841e-04, 0.00000000e+00, 7.06630522e-03, 0.00000000e+00,
        9.73755519e-01],
       [6.11653160e-01, 2.30624250e-02, 0.00000000e+00, 0.00000000e+00,
        9.92211559e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        4.66656632e-02],
       [3.99860972e-01, 0.00000000e+00, 5.24774660e-01, 6.17481510e-01,
        0.00000000e+00]])

### densify -


In [ ]:
#| exporti
class DensifyRules(RuleTable):
    """|RuleTable| for the :func:`densify` algorithm."""

    def __init__(self):
        super().__init__(use_caching=True)

    @match_class(XarrayMatrixOperator)
    def action_XarrayMatrixOperator(self, op):
        op = self.replace_children(op)
        if op.sparse:
            matrix = op.matrix.copy()
            matrix.data = matrix.data.todense()
            op = op.with_(matrix=matrix)
        return op
    
    @match_class(Model, Operator)
    def action_recurse(self, op):
        return self.replace_children(op)

In [ ]:
#| export
def densify(
    obj:Model|Operator # Object to densify
)->Model|Operator: # Densified object
    """Convert sparse operators to dense."""
    return DensifyRules().apply(obj)

An operator backed by a sparse `DataArray`:

In [ ]:
op.sparse

True

Convert the operator to one backed by a dense `DataArray`:

In [ ]:
densify(op).sparse

False

In [ ]:
#|hide
test_eq(_, False)

## XarrayFunctionalOperator

In [ ]:
#| export
class XarrayFunctionalOperator(XarrayMatrixBasedOperator):
    """An `Operator` described by a `ParameterFunctional` that assembles to a `XarrayMatrixOperator`."""
    
    def __init__(self, functional:ParameterFunctional, range, source):
        self.__auto_init(locals())

    def assemble(self, mu=None): 
        return XarrayMatrixOperator(self.functional.evaluate(mu), range=self.range, source=self.source)

    @property
    def size(self):
        return self.matrix.size

## SumOperator

In [ ]:
#| export
class SumOperator(Operator):
    """An `Operator` that sums over one or more dimensions of a `XarrayVectorSpace`."""
    
    linear = True
    
    def __init__(
        self, 
        source_coords:Coordinates|dict, # Coordinates to sum over 
        name=None # Operator name
    ):
        self.__auto_init(locals())
        self.source = XarrayVectorSpace(source_coords)
        self.range = XarrayVectorSpace()

    def assemble(self, mu=None): return self

    def apply(self, U, mu=None):
        U = self.range.from_xarray(U.array.sum(self.source.dims))
        if self.name != self.__class__.__name__: U = U.rename(self.name)
        return U

    def __mul__(self, other):
        if isinstance(other, Operator):
            return ProductOperator([self, other])
        return super().__mul__(other)

    def __rmul__(self, other):
        if isinstance(other, Operator):
            return ProductOperator([other, self])
        return super().__mul__(other)

    def __str__(self):
        return f"{self.name if self.name != self.__class__.__name__ else '∑'}{_mapping_str(self)}"

    def _repr_html_(self):
        return str(self)

In [ ]:
space = XarrayVectorSpace({'A': [1, 2], 'B': ['a', 'b', 'c']}, name='foo')

In [ ]:
np.random.seed(42)
U = space.from_numpy(np.random.rand(space.dim))

In [ ]:
U.array.A

<xarray.DataArray 'A' (A: 2)> Size: 16B
array([1, 2])
Coordinates:
  * A        (A) int64 16B 1 2

In [ ]:
op = SumOperator(U.array.A)
op

SumOperator(
    <xarray.DataArray 'A' (A: 2)> Size: 16B
    array([1, 2])
    Coordinates:
      * A        (A) int64 16B 1 2)

In [ ]:
op.source

XarrayVectorSpace(
    coords=Coordinates:
             * A        (A) int64 16B 1 2,
    name='A')

In [ ]:
op.range

XarrayVectorSpace()

In [ ]:
op.apply(U).array

<xarray.DataArray (B: 3)> Size: 24B
array([0.9731986 , 1.10673295, 0.88798846])
Coordinates:
  * B        (B) <U1 12B 'a' 'b' 'c'

## ScaleOperator

In [ ]:
#| export
class ScaleOperator(Operator):
    """A scaling operator for `XarrayVectorSpace`s."""
    
    def __init__(self, array, space=None, name=None):
        if not isinstance(array, DataArray): array = DataArray(array, coords=space.coords, name=name)
        if space is None: space = XarrayVectorSpace(array)
        self.__auto_init(locals())
        self.source = space
        self.range = space

    def assemble(self, mu=None): return self

    def apply(self, U, mu=None):
        U = U * self.array
        if self.name != self.__class__.__name__: U = U.rename(self.name)
        return U

    @property
    def display_name(self):
        return self.name if self.name != self.__class__.__name__ else 'S'
    
    def __mul__(self, other):
        if isinstance(other, Operator):
            return ProductOperator([self, other])
        return super().__mul__(other)

    def __rmul__(self, other):
        if isinstance(other, Operator):
            return ProductOperator([other, self])
        return super().__mul__(other)

    def __str__(self):
        return f"{self.display_name}{_mapping_str(self)}"

    def _repr_html_(self):
        return str(self)

In [ ]:
space = XarrayVectorSpace({'A': [1, 2], 'B': ['a', 'b', 'c']}, name='foo')

In [ ]:
U = space.from_numpy(np.arange(6))

In [ ]:
U.array

<xarray.DataArray (A: 2, B: 3)> Size: 48B
array([[0, 1, 2],
       [3, 4, 5]])
Coordinates:
  * A        (A) int64 16B 1 2
  * B        (B) <U1 12B 'a' 'b' 'c'

In [ ]:
arr = U.array[0].drop_vars('A')
arr

<xarray.DataArray (B: 3)> Size: 24B
array([0, 1, 2])
Coordinates:
  * B        (B) <U1 12B 'a' 'b' 'c'

In [ ]:
scale_op = ScaleOperator(arr, name='bar')
scale_op

ScaleOperator(
    <xarray.DataArray (B: 3)> Size: 24B
    array([0, 1, 2])
    Coordinates:
      * B        (B) <U1 12B 'a' 'b' 'c',
    space=XarrayVectorSpace(
              coords=Coordinates:
                       * B        (B) <U1 12B 'a' 'b' 'c'),
    name='bar')

In [ ]:
scale_op.apply(U)

XarrayVectorArray(
    XarrayVectorSpace(
        coords=Coordinates:
                 * A        (A) int64 16B 1 2
                 * B        (B) <U1 12B 'a' 'b' 'c',
        name='bar'),
    XarrayVectorArrayImpl(
        <xarray.DataArray 'bar' (A: 2, B: 3)> Size: 48B
        array([[ 0,  1,  4],
               [ 0,  4, 10]])
        Coordinates:
          * A        (A) int64 16B 1 2
          * B        (B) <U1 12B 'a' 'b' 'c',
        <xarray.DataArray 'bar' (A: 2, B: 3)> Size: 48B
        array([[0., 0., 0.],
               [0., 0., 0.]])
        Coordinates:
          * A        (A) int64 16B 1 2
          * B        (B) <U1 12B 'a' 'b' 'c'),
    _len=1)

In [ ]:
space.ones().array.data

array([[1., 1., 1.],
       [1., 1., 1.]])

In [ ]:
ScaleOperator(space.ones().array.data, space=space)

ScaleOperator(
    <xarray.DataArray (A: 2, B: 3)> Size: 48B
    array([[1., 1., 1.],
           [1., 1., 1.]])
    Coordinates:
      * A        (A) int64 16B 1 2
      * B        (B) <U1 12B 'a' 'b' 'c',
    space=XarrayVectorSpace(
              coords=Coordinates:
                       * A        (A) int64 16B 1 2
                       * B        (B) <U1 12B 'a' 'b' 'c',
              name='foo'))

In [ ]:
#| hide
# from pylgs.pymor.example_problems import *

# A, B, C, E = lti_matrices()

# input_space = XarrayVectorSpace({'inputs': ['uniform heating', 'left end heating']}, name='input')

# U = input_space.from_numpy(np.array([.3, .7]))

# U

# matrix=DataArray(B, {'grid': np.linspace(0, 1, 101), 'inputs': U.array.coords['inputs']})

# op = XarrayMatrixOperator(matrix)

# op

# op.apply(U)

# op.range

# op.source

# op.H.source

# op.as_range_array()

# op.as_source_array()

# V=op.range.from_numpy(np.linspace(0, 2, 101))

# op.apply_adjoint(V)

## LincombOperator

In [ ]:
#| export
def _no_space_str(op):
    if isinstance(op, XarrayMatrixOperator): return op.name if op.name != op.__class__.__name__ else 'X'
    if isinstance(op, LincombOperator): return '[' + ' + '.join(f'{_no_space_str(c)}·{_no_space_str(o)}' for c, o in zip(op.coefficients, op.operators)) + ']'
    return str(op)

In [ ]:
#| export
@patch
def __str__(self:LincombOperator):
    return f"{_no_space_str(self)}{_mapping_str(self)}"

@patch
def _repr_html_(self:LincombOperator):
    return str(self)

### _numpy_zero_operator -


In [ ]:
#| export
def _numpy_zero_operator(shape):
    return ZeroOperator(*[NumpyVectorSpace(dim) for dim in shape])

### read_sparse_dataarray -


In [ ]:
#| export
def _read_sparse_dataarray(
    file_name:str|Path, # File name
)->DataArray:
    if not Path(file_name).suffix: file_name = Path(file_name).with_suffix('.mtxn')
    header = []
    with open(file_name, "r") as reader:
        while True:
            line = next(reader)
            if line[0] != "%": break
            header.append(line.strip('%\n '))
    matrix = pd.read_csv(
        file_name,
        sep='\s+',
        header=None,
        skiprows=len(header),
        index_col=None,
        engine="c",
        comment='%'
    )
    name = header[0]
    parameters = header[1].split('\t')
    shape = [int(s) for s in header[2].split()]
    coords = {dim: index.split('\t') for dim, index in zip(header[3::2], header[4::2])}
    matrix = sparse([matrix.iloc[:, :-1].T, matrix.iloc[:, -1]], shape=shape)
    return DataArray(matrix, coords, name=name, attrs={'parameters': parameters})

In [ ]:
_read_sparse_dataarray('test')

Format,coo
Data Type,float64
Shape,"(2, 3, 4)"
nnz,2
Density,0.08333333333333333
Read-only,True
Size,64
Storage ratio,0.33


In [ ]:
with importlib.resources.path("pylgs.systems.NaD1_Toy", "Flux.mtxn") as path:
    out = _read_sparse_dataarray(path)
out

Format,coo
Data Type,float64
Shape,"(1, 1, 4)"
nnz,1
Density,0.25
Read-only,True
Size,32
Storage ratio,1.00


### from_file -


In [ ]:
#| export 
@patch(cls_method=True)
@delegates(LincombOperator.__init__)
def from_file(
    cls:LincombOperator, 
    file_name:str|Path, # File name
    **kwargs # Additional arguments are passed to `LincombOperator`.
):
    """Read a `LincombOperator` (list of sparse arrays and corresponding `ExpressionParameterFunctionals`) from a .mtxn file."""
    da = _read_sparse_dataarray(file_name)
    coefficients = [
        ExpressionParameterFunctional(s, {p: 1 for p in da.parameters if p in s}) if da.parameters != [''] else 1
        for s in da.coords[da.dims[0]].data
    ]
    del da.attrs['parameters']
    operators = [XarrayMatrixOperator(d.drop_vars(da.dims[0])) for d in da]    
    return LincombOperator(operators, coefficients, **kwargs)

In [ ]:
with importlib.resources.path("pylgs.systems.NaD1_Toy", "Flux.mtxn") as path:
    op = LincombOperator.from_file(path)
op

LincombOperator(
    (XarrayMatrixOperator(
         <xarray.DataArray 'Flux' (Transition: 1, Density matrix (source): 4)> Size: 24B
         <COO: shape=(1, 4), dtype=float64, nnz=1, fill_value=0.0>
         Coordinates:
           * Transition               (Transition) <U33 132B '3P<sub>1/2</sub>→3S<sub>...
           * Density matrix (source)  (Density matrix (source)) <U50 800B 'ρ<sub>Re, 3...,
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * Density matrix (source)  (Density matrix (source)) <U50 800B 'ρ<sub>Re, 3...),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * Transition  (Transition) <U33 132B '3P<sub>1/2</sub>→3S<sub>1/2</sub>',
                   name='Flux'),
         name='Flux')),
    (1))

In [ ]:
op.assemble()

XarrayMatrixOperator(
    <xarray.DataArray 'Flux' (Transition: 1, Density matrix (source): 4)> Size: 24B
    <COO: shape=(1, 4), dtype=float64, nnz=1, fill_value=0.0>
    Coordinates:
      * Transition               (Transition) <U33 132B '3P<sub>1/2</sub>→3S<sub>...
      * Density matrix (source)  (Density matrix (source)) <U50 800B 'ρ<sub>Re, 3...,
    source=XarrayVectorSpace(
               coords=Coordinates:
                        * Density matrix (source)  (Density matrix (source)) <U50 800B 'ρ<sub>Re, 3...),
    range=XarrayVectorSpace(
              coords=Coordinates:
                       * Transition  (Transition) <U33 132B '3P<sub>1/2</sub>→3S<sub>1/2</sub>',
              name='Flux'),
    name='Flux')

In [ ]:
op = LincombOperator.from_file('test.mtxn')
op

LincombOperator(
    (XarrayMatrixOperator(
         <xarray.DataArray 'test' (another dim: 3, third dim: 4)> Size: 24B
         <COO: shape=(3, 4), dtype=float64, nnz=1, fill_value=0.0>
         Coordinates:
           * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz'
           * third dim    (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$',
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * third dim  (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$'),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz',
                   name='test'),
         name='test'),
     XarrayMatrixOperator(
         <xarray.DataArray 'test' (another dim: 3, third dim: 4)> Size: 24B
         <COO: shape=(3, 4), dtype=float64, nnz=1, fill_value=0.0>
         Coordinates:
           * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz'
           * third dim    (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$',
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * third dim  (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$'),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz',
                   name='test'),
         name='test')),
    (ExpressionParameterFunctional('A**2', {A: 1}), ExpressionParameterFunctional('A*sin(B)', {A: 1, B: 1})))

In [ ]:
op.source

XarrayVectorSpace(
    coords=Coordinates:
             * third dim  (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$')

In [ ]:
op.range

XarrayVectorSpace(
    coords=Coordinates:
             * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz',
    name='test')

### partial_evaluate -


In [ ]:
#| export 
@patch
def partial_evaluate(
    self:ExpressionParameterFunctional, 
    mu:dict|Mu # Parameter values 
)->float|ExpressionParameterFunctional:
    """Substitute parameter values, returning a new `ExpressionParameterFunctional` if expression does not evaluate to a number."""
    mu = {k: v[0] for k, v in mu.items()}
    expr = sy.sympify(self.expression).subs(mu)
    unevaluated = [str(s) for s in expr.free_symbols]
    if expr.is_number:
        return float(expr)
    return ExpressionParameterFunctional(
        str(expr), 
        {k: v for k, v in self.parameters.items() if k in unevaluated}
    )

In [ ]:
op.coefficients[1].partial_evaluate({'A': [.5]})

ExpressionParameterFunctional('0.5*sin(B)', {B: 1})

In [ ]:
op.coefficients[1].partial_evaluate({'A': [.5], 'B': [2.]})

0.45464871341284085

### partial_evaluate_coefficients -


In [ ]:
#| export 
@patch
def partial_evaluate_coefficients(
    self:LincombOperator, 
    mu:Mu # Parameter values to substitute
):
    """Substitute parameter values into linear coefficients, returning a new `ExpressionParameterFunctional` if expression does not evaluate to a number."""
    result = []
    for c in self.coefficients:
        try:
            result.append(c.evaluate(mu))
        except AttributeError:
            result.append(c)
        except AssertionError:
            try:
                result.append(c.partial_evaluate(mu))
            except Exception as e:
                result.append(c)
    return result

In [ ]:
op.partial_evaluate_coefficients(Mu({'A': .5}))

[0.25, ExpressionParameterFunctional('0.5*sin(B)', {B: 1})]

### partial_assemble -


In [ ]:
#| export 
@patch
def partial_assemble(
    self:LincombOperator, 
    mu=None # Parameter values to substitute
)->Operator:
    """Substitute parameter values into the linear coefficients, returning a new operator with fewer (or no) parameters."""
    operators = tuple(op.assemble(mu) for op in self.operators)
    coefficients = self.partial_evaluate_coefficients(mu)
    evaluated = [[], []]
    unevaluated = [[], []]
    for o, c in zip(operators, coefficients):
        if isinstance(c, Number):
            if c:
                evaluated[0].append(o)
                evaluated[1].append(c)
        else:
            unevaluated[0].append(o)
            unevaluated[1].append(c)
    op = []
    if len(evaluated[0]):
        op.append(assemble_lincomb(
            *evaluated, 
            solver_options=self.solver_options,
            name=self.name + '_partially_assembled'
        ))
    if len(unevaluated[0]):
        op.append(LincombOperator(*unevaluated))
    if len(op) == 0:
        op = [ZeroOperator(range=self.range, source=self.source)]
    op = np.sum(op)
    # To avoid infinite recursions, only use the result if at least one of the following
    # is true:
    #   - The operator is parametric, so the the result of assemble *must* be a different,
    #     non-parametric operator.
    #   - One of self.operators changed by calling 'assemble' on it.
    #   - The result of assemble_lincomb is of a different type than the original operator.
    #   - assemble_lincomb could simplify the list of assembled operators,
    #     which we define to be the case when the number of operators has ben reduced.
    if (self.parametric
            or operators != self.operators  # for this comparison to work self.operators always has to be a tuple!
            or type(op) != type(self)
            or len(op.operators) < len(operators)):
        return op
    else:
        return self

In [ ]:
op

LincombOperator(
    (XarrayMatrixOperator(
         <xarray.DataArray 'test' (another dim: 3, third dim: 4)> Size: 24B
         <COO: shape=(3, 4), dtype=float64, nnz=1, fill_value=0.0>
         Coordinates:
           * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz'
           * third dim    (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$',
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * third dim  (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$'),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz',
                   name='test'),
         name='test'),
     XarrayMatrixOperator(
         <xarray.DataArray 'test' (another dim: 3, third dim: 4)> Size: 24B
         <COO: shape=(3, 4), dtype=float64, nnz=1, fill_value=0.0>
         Coordinates:
           * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz'
           * third dim    (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$',
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * third dim  (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$'),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz',
                   name='test'),
         name='test')),
    (ExpressionParameterFunctional('A**2', {A: 1}), ExpressionParameterFunctional('A*sin(B)', {A: 1, B: 1})))

In [ ]:
op.partial_assemble(Mu({'A': .5}))

LincombOperator(
    (XarrayMatrixOperator(
         <xarray.DataArray 'test' (another dim: 3, third dim: 4)> Size: 24B
         <COO: shape=(3, 4), dtype=float64, nnz=1, fill_value=0.0>
         Coordinates:
           * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz'
           * third dim    (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$',
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * third dim  (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$'),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz',
                   name='test'),
         name='test'),
     XarrayMatrixOperator(
         <xarray.DataArray 'test' (another dim: 3, third dim: 4)> Size: 24B
         <COO: shape=(3, 4), dtype=float64, nnz=1, fill_value=0.0>
         Coordinates:
           * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz'
           * third dim    (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$',
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * third dim  (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$'),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz',
                   name='test'),
         name='test')),
    (1.0, ExpressionParameterFunctional('0.5*sin(B)', {B: 1})))

In [ ]:
op.partial_assemble(Mu({'A': .5, 'B': 2.}))

XarrayMatrixOperator(
    <xarray.DataArray 'test' (another dim: 3, third dim: 4)> Size: 48B
    <COO: shape=(3, 4), dtype=float64, nnz=2, fill_value=0.0>
    Coordinates:
      * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz'
      * third dim    (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$',
    source=XarrayVectorSpace(
               coords=Coordinates:
                        * third dim  (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$'),
    range=XarrayVectorSpace(
              coords=Coordinates:
                       * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz',
              name='test'),
    name='test')

In [ ]:
op.partial_assemble(Mu({'A': 0}))

ZeroOperator(
    XarrayVectorSpace(
        coords=Coordinates:
                 * another dim  (another dim) <U3 36B 'foo' 'bar' 'baz',
        name='test'),
    XarrayVectorSpace(
        coords=Coordinates:
                 * third dim  (third dim) <U3 48B '$a$' '$b$' '$c$' '$d$'))

### terms -


In [ ]:
#| export
@patch(as_prop=True)
def terms(self:LincombOperator):
    return [c * o for c, o in zip(self.coefficients, self.operators)]

In [ ]:
# #| export
# @patch
# def scalar_part(self:LincombOperator, mu):
#     mu_vector = Mu({k: v for k, v in mu.items() if v.size > 1})
#     return np.sum([o for o in self.terms if not set(mu_vector).intersection(o.parameters)])

In [ ]:
# #| export
# @patch
# def vector_part(self:LincombOperator, mu):
#     mu_vector = Mu({k: v for k, v in mu.items() if v.size > 1})
#     return np.sum([o for o in self.terms if set(mu_vector).intersection(o.parameters)])

## ProductOperator

In [ ]:
#| export 
class ProductOperator(Operator):
    """An Operator given by the direct product of operators `operators`."""
    def __init__(
        self, 
        operators, # Sequence of operators that each assemble to XarrayMatrixOperator
        name=None, # Optional name
    ):
        for op in operators:
            if not isinstance(op, Operator): raise ValueError(f'Input {op} should be an Operator.')
        self.source = np.product([op.source for op in operators])
        self.range = np.product([op.range for op in operators])
        self.__auto_init(locals())
        self.linear = True
        
    def apply(self, U, mu=None):
        pass

In [ ]:
A = XarrayMatrixOperator(array([[1, 0], [0, 1]]), source=['source a'], range=['range a'])
B = XarrayMatrixOperator(array([[0, 1], [1, 0]]), source=['source b'], range=['range b'])

In [ ]:
#| export
@patch
def __mul__(self:Operator, other):
    if isinstance(other, Operator) and all(isinstance(space, XarrayVectorSpace) for space in [self.source, self.range, other.source, other.range]):
        return ProductOperator([self, other])
    assert isinstance(other, (Number, ParameterFunctional))
    # from pymor.operators.constructions import LincombOperator
    if self.name != 'LincombOperator' or not isinstance(self, LincombOperator):
        return LincombOperator((self,), (other,))
    else:
        return self.with_(coefficients=tuple(c * other for c in self.coefficients))

In [ ]:
#| export
@patch
def __str__(self:ProductOperator):
    return ' ⨂ '.join(str(o) for o in self.operators)

In [ ]:
#| export
@patch
def _repr_html_(self:ProductOperator):
    return str(self)

In [ ]:
#| export
@patch
def __str__(self:IdentityOperator):
    return f'I{_mapping_str(self)}'

@patch
def _repr_html_(self:IdentityOperator):
    return str(self)

In [ ]:
A * IdentityOperator(B.source)

ProductOperator(
    [XarrayMatrixOperator(
         <xarray.DataArray (range a: 2, source a: 2)> Size: 32B
         array([[1, 0],
                [0, 1]])
         Dimensions without coordinates: range a, source a,
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * source a  (source a) int64 16B 0 1),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * range a  (range a) int64 16B 0 1)),
     IdentityOperator(
         XarrayVectorSpace(
             coords=Coordinates:
                      * source b  (source b) int64 16B 0 1))])

In [ ]:
IdentityOperator(B.source) * A

ProductOperator(
    [IdentityOperator(
         XarrayVectorSpace(
             coords=Coordinates:
                      * source b  (source b) int64 16B 0 1)),
     XarrayMatrixOperator(
         <xarray.DataArray (range a: 2, source a: 2)> Size: 32B
         array([[1, 0],
                [0, 1]])
         Dimensions without coordinates: range a, source a,
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * source a  (source a) int64 16B 0 1),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * range a  (range a) int64 16B 0 1))])

In [ ]:
def vec(a):
    return a.T.ravel()

In [ ]:
s=1

In [ ]:
A = XarrayMatrixOperator(
    sparse2d_rand([3*s, 2*s], density=.1, random_state=42), 
    source='source a',
    range='range a'
)
A

XarrayMatrixOperator(
    <xarray.DataArray (range a: 3, source a: 2)> Size: 24B
    <COO: shape=(3, 2), dtype=float64, nnz=1, fill_value=0.0>
    Dimensions without coordinates: range a, source a,
    source=XarrayVectorSpace(
               coords=Coordinates:
                        * source a  (source a) int64 16B 0 1),
    range=XarrayVectorSpace(
              coords=Coordinates:
                       * range a  (range a) int64 24B 0 1 2))

In [ ]:
mu = Mu(A=.5)

In [ ]:
B = ExpressionParameterFunctional('A**2', {'A': 1}) * XarrayMatrixOperator(
    sparse2d_rand([4*s, 5*s], density=.1, random_state=42),
    source='source b',
    range='range b'
)
B = B.assemble(mu)

In [ ]:
op = A * B
op

ProductOperator(
    [XarrayMatrixOperator(
         <xarray.DataArray (range a: 3, source a: 2)> Size: 24B
         <COO: shape=(3, 2), dtype=float64, nnz=1, fill_value=0.0>
         Dimensions without coordinates: range a, source a,
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * source a  (source a) int64 16B 0 1),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * range a  (range a) int64 24B 0 1 2)),
     XarrayMatrixOperator(
         <xarray.DataArray (range b: 4, source b: 5)> Size: 48B
         <COO: shape=(4, 5), dtype=float64, nnz=2, fill_value=0.0>
         Dimensions without coordinates: range b, source b,
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * source b  (source b) int64 40B 0 1 2 3 4),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * range b  (range b) int64 32B 0 1 2 3))])

In [ ]:
op.source

XarrayVectorSpace(
    coords=Coordinates:
             * source a  (source a) int64 16B 0 1
             * source b  (source b) int64 40B 0 1 2 3 4)

In [ ]:
op.range

XarrayVectorSpace(
    coords=Coordinates:
             * range a  (range a) int64 24B 0 1 2
             * range b  (range b) int64 32B 0 1 2 3)

In [ ]:
B.source._array

<xarray.DataArray (source b: 5)> Size: 40B
array([0., 0., 0., 0., 0.])
Coordinates:
  * source b  (source b) int64 40B 0 1 2 3 4

In [ ]:
sum_op = SumOperator(B.source._array['source b'])

In [ ]:
sum_op * A

ProductOperator(
    [SumOperator(
         <xarray.DataArray 'source b' (source b: 5)> Size: 40B
         array([0, 1, 2, 3, 4])
         Coordinates:
           * source b  (source b) int64 40B 0 1 2 3 4),
     XarrayMatrixOperator(
         <xarray.DataArray (range a: 3, source a: 2)> Size: 24B
         <COO: shape=(3, 2), dtype=float64, nnz=1, fill_value=0.0>
         Dimensions without coordinates: range a, source a,
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * source a  (source a) int64 16B 0 1),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * range a  (range a) int64 24B 0 1 2))])

In [ ]:
A * sum_op

ProductOperator(
    [XarrayMatrixOperator(
         <xarray.DataArray (range a: 3, source a: 2)> Size: 24B
         <COO: shape=(3, 2), dtype=float64, nnz=1, fill_value=0.0>
         Dimensions without coordinates: range a, source a,
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * source a  (source a) int64 16B 0 1),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * range a  (range a) int64 24B 0 1 2)),
     SumOperator(
         <xarray.DataArray 'source b' (source b: 5)> Size: 40B
         array([0, 1, 2, 3, 4])
         Coordinates:
           * source b  (source b) int64 40B 0 1 2 3 4)])

In [ ]:
np.random.seed(42)
U = op.source.from_numpy(np.random.rand(op.source._array.size))
V = op.range.from_numpy(np.random.rand(op.range._array.size))

In [ ]:
U.array

<xarray.DataArray (source a: 2, source b: 5)> Size: 80B
array([[0.37454012, 0.95071431, 0.73199394, 0.59865848, 0.15601864],
       [0.15599452, 0.05808361, 0.86617615, 0.60111501, 0.70807258]])
Coordinates:
  * source a  (source a) int64 16B 0 1
  * source b  (source b) int64 40B 0 1 2 3 4

In [ ]:
U.space

XarrayVectorSpace(
    coords=Coordinates:
             * source a  (source a) int64 16B 0 1
             * source b  (source b) int64 40B 0 1 2 3 4)

In [ ]:
An = A.matrix.data
Bn = B.assemble(mu).matrix.data
Un = U.to_numpy().T
Vn = V.to_numpy().T

In [ ]:
%%time 
An = A.matrix.data
Bn = B.assemble(mu).matrix.data
Bn = B.assemble(mu).matrix.data
Un = U.to_numpy().T
out1 = np.kron(An, Bn).dot(vec(Un))

CPU times: user 111 ms, sys: 3.24 ms, total: 114 ms
Wall time: 112 ms


In [ ]:
%%time
An = A.matrix.data
Bn = B.assemble(mu).matrix.data
Bn = B.assemble(mu).matrix.data
Un = U.array.T.data
out3 = An.dot(Bn.dot(Un).T).ravel()

CPU times: user 381 μs, sys: 73 μs, total: 454 μs
Wall time: 416 μs


In [ ]:
%%time
out4 = A.apply(B.apply(U)).to_numpy().T

CPU times: user 75.9 ms, sys: 2.05 ms, total: 77.9 ms
Wall time: 76.7 ms


In [ ]:
test_close(out1.ravel(), out3.ravel())
test_close(out1.ravel(), out4.ravel())

In [ ]:
test_close(
    np.kron(An, Bn).dot(vec(Un)),
    An.dot(Bn.dot(Un).T).ravel()
)

 N.B.: `to_numpy` on a `NumpyVectorArray` effectively takes the transpose.

### apply -


In [ ]:
#| export 
@patch
def apply(self:ProductOperator, U, mu=None):
    U = U.copy()
    for op in self.operators[::-1]:
        U = op.assemble(mu).apply(U)
    return self.range.from_xarray(U.array)

In [ ]:
W = space.from_numpy(np.arange(12))

In [ ]:
C = ScaleOperator(W.array[0, 0].drop_vars(['A', 'len']), name='C')
C.apply(W).name

'C'

In [ ]:
#| hide
test_eq(_, 'C')

In [ ]:
D = SumOperator(W.array.A, name='D')
D.apply(W).name

'D'

In [ ]:
#| hide
test_eq(_, 'D')

In [ ]:
C = ScaleOperator(W.array[0, 0].drop_vars(['A', 'len']))
E = (C * D)
E.apply(W).name

'D'

In [ ]:
#| hide
test_eq(_, 'D')

In [ ]:
E = C * (C * D)
E.apply(W).array.name

'D'

In [ ]:
#| hide
test_eq(_, 'D')

In [ ]:
%%time 
K = A * B.assemble(mu)
out2 = K.apply(U).to_numpy()

CPU times: user 2.49 ms, sys: 108 μs, total: 2.6 ms
Wall time: 2.54 ms


In [ ]:
# test_close(out1.ravel(), out2.ravel())

In [ ]:
test_close(
    op.apply(U, mu).to_numpy().ravel(), 
    np.kron(An, Bn).dot(vec(Un))
)

$(A\bigotimes B)^T=A^T\bigotimes B^T$

In [ ]:
test_close(np.kron(An, Bn).T, np.kron(An.T, Bn.T))

### H -


In [ ]:
#| export 
@patch(as_prop=True)
def H(self:ProductOperator):
    return self.with_(operators=[op.H for op in self.operators])

In [ ]:
op.H

ProductOperator(
    [XarrayMatrixOperator(
         <xarray.DataArray (range a: 3, source a: 2)> Size: 24B
         <COO: shape=(3, 2), dtype=float64, nnz=1, fill_value=0.0>
         Dimensions without coordinates: range a, source a,
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * range a  (range a) int64 24B 0 1 2),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * source a  (source a) int64 16B 0 1,
                   name='XarrayMatrixOperator_adjoint'),
         name='XarrayMatrixOperator_adjoint'),
     XarrayMatrixOperator(
         <xarray.DataArray (range b: 4, source b: 5)> Size: 48B
         <COO: shape=(4, 5), dtype=float64, nnz=2, fill_value=0.0>
         Dimensions without coordinates: range b, source b,
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * range b  (range b) int64 32B 0 1 2 3),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * source b  (source b) int64 40B 0 1 2 3 4,
                   name='XarrayMatrixOperator_adjoint'),
         name='XarrayMatrixOperator_adjoint')])

$\text{vec}(V)(A\bigotimes B)=(A\bigotimes B)^T\text{vec}(V)$.

In [ ]:
op.H.source

XarrayVectorSpace(
    coords=Coordinates:
             * range a  (range a) int64 24B 0 1 2
             * range b  (range b) int64 32B 0 1 2 3)

In [ ]:
test_close(
    vec(Vn).dot(np.kron(An, Bn).todense()),
    (np.kron(An, Bn).T).dot(vec(Vn))
)

### apply_adjoint -


In [ ]:
#| export 
@patch
def apply_adjoint(self:ProductOperator, V, mu=None):
    return self.H.apply(V, mu=mu)

In [ ]:
test_close(
    op.apply_adjoint(V, mu).to_numpy().ravel(),
    vec(Vn).dot(np.kron(An, Bn).todense())
)

### assemble -


In [ ]:
#| export 
@patch
def assemble(self:ProductOperator, mu=None):
    operators = [op.assemble(mu) for op in self.operators]
    if any(isinstance(op, ZeroOperator) for op in operators): return ZeroOperator(self.range, self.source)
    if all(op == old_op for op, old_op in zip(operators, self.operators)): return self 
    return self.with_(operators=operators)

In [ ]:
op.assemble(mu)

ProductOperator(
    [XarrayMatrixOperator(
         <xarray.DataArray (range a: 3, source a: 2)> Size: 24B
         <COO: shape=(3, 2), dtype=float64, nnz=1, fill_value=0.0>
         Dimensions without coordinates: range a, source a,
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * source a  (source a) int64 16B 0 1),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * range a  (range a) int64 24B 0 1 2)),
     XarrayMatrixOperator(
         <xarray.DataArray (range b: 4, source b: 5)> Size: 48B
         <COO: shape=(4, 5), dtype=float64, nnz=2, fill_value=0.0>
         Dimensions without coordinates: range b, source b,
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * source b  (source b) int64 40B 0 1 2 3 4),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * range b  (range b) int64 32B 0 1 2 3))])

In [ ]:
zero = (0 * B).assemble()

In [ ]:
(A * zero).assemble()

ZeroOperator(
    XarrayVectorSpace(
        coords=Coordinates:
                 * range a  (range a) int64 24B 0 1 2
                 * range b  (range b) int64 32B 0 1 2 3),
    XarrayVectorSpace(
        coords=Coordinates:
                 * source a  (source a) int64 16B 0 1
                 * source b  (source b) int64 40B 0 1 2 3 4))

### expand_ProductOperator -


In [ ]:
#| exporti
@match_class(ProductOperator)
def expand_ProductOperator(self, op):
    op = self.replace_children(op)

    # merge child ProductOperators
    if any(isinstance(o, ProductOperator) for o in op.operators):
        ops = []
        for o in op.operators:
            if isinstance(o, ProductOperator):
                ops.extend(o.operators)
            else:
                ops.append(o)
        op = op.with_(operators=ops)

    # expand products with LincombOperators
    if any(isinstance(o, LincombOperator) for o in op.operators):
        i = next(iter(i for i, o in enumerate(op.operators) if isinstance(o, LincombOperator)))
        left, right = op.operators[:i], op.operators[i+1:]
        ops = [ProductOperator(left + (o,) + right) for o in op.operators[i].operators]
        op = op.operators[i].with_(operators=ops)

        # there can still be LincombOperators within the summands so we recurse ..
        op = self.apply(op)
    return op

#| export
ExpandRules.insert_rule(-1, expand_ProductOperator)

### contract_ProductOperator -


In [ ]:
#| exporti
@match_class(ProductOperator)
def contract_ProductOperator(self, op):
    op = self.replace_children(op)
    if any(isinstance(o, ZeroOperator) for o in op.operators): return ZeroOperator(op.range, op.source)
    xarray_operators = [o for o in op.operators if     isinstance(o, XarrayMatrixOperator)]
    other_operators  = [o for o in op.operators if not isinstance(o, XarrayMatrixOperator)]
    if xarray_operators:
        matrices, ranges, sources = unzip([(o.matrix, o.range, o.source) for o in xarray_operators])
        op = XarrayMatrixOperator(
            math.prod(matrices), 
            source=math.prod(sources, start=XarrayVectorSpace()), 
            range=math.prod(ranges, start=XarrayVectorSpace())
        )
    else:
        op = 1
    if other_operators:
        op *= math.prod(other_operators) 
    return op

ContractRules.insert_rule(-2, contract_ProductOperator)

In [ ]:
#| hide
contract(op)

XarrayMatrixOperator(
    <xarray.DataArray (range a: 3, source a: 2, range b: 4, source b: 5)> Size: 80B
    <COO: shape=(3, 2, 4, 5), dtype=float64, nnz=2, fill_value=0.0>
    Dimensions without coordinates: range a, source a, range b, source b,
    source=XarrayVectorSpace(
               coords=Coordinates:
                        * source a  (source a) int64 16B 0 1
                        * source b  (source b) int64 40B 0 1 2 3 4),
    range=XarrayVectorSpace(
              coords=Coordinates:
                       * range a  (range a) int64 24B 0 1 2
                       * range b  (range b) int64 32B 0 1 2 3))

### to_matrix -

In [ ]:
# #| hide
# #| exporti
# @match_class(ProductOperator)
# def to_matrix_ProductOperator(self, op):
#     return self.apply(contract(op))
    
# ToMatrixRules.insert_rule(-2, to_matrix_ProductOperator)

In [ ]:
# #| hide
# to_matrix(op, mu=mu)

In [ ]:
#| hide
#| exporti
@match_class(ProductOperator)
def to_matrix_ProductOperator(self, op):
    return kron(*[self.apply(o) for o in op.operators])
    
ToMatrixRules.insert_rule(-2, to_matrix_ProductOperator)

In [ ]:
to_matrix(op, mu=mu)

<Compressed Sparse Row sparse array of dtype 'float64'
	with 2 stored elements and shape (12, 10)>

In [ ]:
test_array('operators', 'to_matrix_ProductOperator_1', _.todense())

### as_source_array -


In [ ]:
#| export 
@patch
def as_source_array(self:ProductOperator, mu=None):
    return contract(self.assemble(mu)).as_source_array()

In [ ]:
op.as_source_array(mu)

XarrayVectorArray(
    XarrayVectorSpace(
        coords=Coordinates:
                 * source a  (source a) int64 16B 0 1
                 * source b  (source b) int64 40B 0 1 2 3 4),
    XarrayVectorArrayImpl(
        <xarray.DataArray (range a: 3, source a: 2, range b: 4, source b: 5)> Size: 80B
        <COO: shape=(3, 2, 4, 5), dtype=float64, nnz=2, fill_value=0.0>
        Dimensions without coordinates: range a, source a, range b, source b,
        <xarray.DataArray (source a: 2, source b: 5)> Size: 80B
        array([[0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.]])
        Coordinates:
          * source a  (source a) int64 16B 0 1
          * source b  (source b) int64 40B 0 1 2 3 4),
    _len=12)

In [ ]:
test_close(
    np.kron(An, Bn).todense().ravel(),
    op.as_source_array(mu).to_numpy().ravel()
)

### as_range_array -


In [ ]:
#| export 
@patch
def as_range_array(self:ProductOperator, mu=None):
    return contract(self.assemble(mu)).as_range_array()

In [ ]:
test_close(
    np.kron(An, Bn).todense().ravel(),
    op.as_range_array(mu).to_numpy().T.ravel()
)

In [ ]:
op.as_range_array(mu)

XarrayVectorArray(
    XarrayVectorSpace(
        coords=Coordinates:
                 * range a  (range a) int64 24B 0 1 2
                 * range b  (range b) int64 32B 0 1 2 3),
    XarrayVectorArrayImpl(
        <xarray.DataArray (range a: 3, source a: 2, range b: 4, source b: 5)> Size: 80B
        <COO: shape=(3, 2, 4, 5), dtype=float64, nnz=2, fill_value=0.0>
        Dimensions without coordinates: range a, source a, range b, source b,
        <xarray.DataArray (range a: 3, range b: 4)> Size: 96B
        array([[0., 0., 0., 0.],
               [0., 0., 0., 0.],
               [0., 0., 0., 0.]])
        Coordinates:
          * range a  (range a) int64 24B 0 1 2
          * range b  (range b) int64 32B 0 1 2 3),
    _len=10)

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()